# -영진위 api 크롤링

http://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=f5eef3421c602c6cb7ea224104795888&targetDt=20120101

- 위 주소를 보면 파라미터로 key와 targetDt가 있는 걸 볼 수 있다



- 여기서 key는 회원 가입을 하면 발급해주는 개인 식별용 키입니다.
    - 키를 발급해주는 이유는 일일 요청 횟수를 제한해서, 서버 메모리를 절약하기 위함입니다.
    - 키를 발급 안하면 디도스 공격을 받을수도?
    - 누가 하루에 몇 번까지 들어올 수 있는지



- targetDt는 타겟 날짜로, 현재 이 api 요청 주소는 일일 박스오피스입니다.
    - 위의 주소를 인터넷 주소창에 복붙하면 날짜 20120101의 10위까지 확인할 수 있다.
    - 검색 키워드 : KOBIS(영화관입장권통합전산망)
    - 검색 키워드 : http://kobis.or.kr/kobisopenapi > OPEN API 클릭 > 일별 박스오피스, 주간/주말 박스오피등 등.. 확인 가능
    
    
    
- http://kobis.or.kr/kobisopenapi/homepg/apiservice/searchServiceInfo.do > 주소를 클릭하여 '응답구조' 자세한 정보 확인하기
    - 받은 제이슨 데이터의 설명들을 확인할 수 있음

- api 서버 접근시, 파이썬에서는 urllib.request를 활용합니다.(requests와는 다른 것)

In [1]:
# 1. 사이트에 자료 요청용 라이브러리 임포트
import urllib.request

# 2. json 데이터 핸들링용
import json
import xml #안해도 됨

In [26]:
# 3. 요청 주소의 파라미터는 다 분해해서 포매팅 형식으로 관리하면 효율적입니다.

key = "f5eef3421c602c6cb7ea224104795888"
target_dt = "20130302"

request_url = "http://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/\
searchDailyBoxOfficeList.json?key=%s&targetDt=%s" %(key, target_dt)

print(request_url)

http://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=f5eef3421c602c6cb7ea224104795888&targetDt=20130302


In [27]:
# 4. 위에 세팅된 주소로 요청을 넣어보겠습니다

result = urllib.request.urlopen(request_url)

In [28]:
# 5. 결과물은 http.client.httpresponse 객체로 주어집니다
result

In [29]:
# 6. 얻어진 데이터의 json만 뽑기 위해 .read()를 사용합니다.
json_raw = result.read()

In [30]:
# 7. 확인

print(json_raw)

b'{"boxOfficeResult":{"boxofficeType":"\xec\x9d\xbc\xeb\xb3\x84 \xeb\xb0\x95\xec\x8a\xa4\xec\x98\xa4\xed\x94\xbc\xec\x8a\xa4","showRange":"20130302~20130302","dailyBoxOfficeList":[{"rnum":"1","rank":"1","rankInten":"0","rankOldAndNew":"OLD","movieCd":"20124401","movieNm":"\xec\x8b\xa0\xec\x84\xb8\xea\xb3\x84","openDt":"2013-02-21","salesAmt":"2302392559","salesShare":"30.0","salesInten":"-93876032","salesChange":"-3.9","salesAcc":"17105407905","audiCnt":"294841","audiInten":"-12964","audiChange":"-4.2","audiAcc":"2283354","scrnCnt":"604","showCnt":"2707"},{"rnum":"2","rank":"2","rankInten":"0","rankOldAndNew":"OLD","movieCd":"20127593","movieNm":"7\xeb\xb2\x88\xeb\xb0\xa9\xec\x9d\x98 \xec\x84\xa0\xeb\xac\xbc","openDt":"2013-01-23","salesAmt":"1943597500","salesShare":"25.3","salesInten":"-222198500","salesChange":"-10.3","salesAcc":"82184894170","audiCnt":"265741","audiInten":"-33121","audiChange":"-11.1","audiAcc":"11491601","scrnCnt":"577","showCnt":"2479"},{"rnum":"3","rank":"3","ra

- (문제) 한국어 데이터가 깨지는 경우
    - type(json_raw) 검사를 하면 <class 'bytes'>라고 되어있다. (한글을 바이트 단위로 쪼개어 표현)
    - 바이트 자료형에는 출력했을 때 무조건 b' 가 붙어있다.
    - str을 붙여서 type이 str이 된다고 해도, 한글로 바뀌지는 않는다.


- (해결) 한글로 돌리는 절차가 필요하다.
    - 형식을 한글로 쓸 수 있는 utf-8 형식으로 고쳐야 합니다. 변형 -> encode(), 복원 -> decode()
    - encode => 우리가 쓰는 문자를 컴퓨터가 쓰는 형태로 변환
    - decode => 컴퓨터가 쓰는 문자를 우리가 쓰는 형태로 변환

In [31]:
# 8. decode하는 코드

json_utf8 = json_raw.decode("utf-8")

print(json_utf8)

{"boxOfficeResult":{"boxofficeType":"일별 박스오피스","showRange":"20130302~20130302","dailyBoxOfficeList":[{"rnum":"1","rank":"1","rankInten":"0","rankOldAndNew":"OLD","movieCd":"20124401","movieNm":"신세계","openDt":"2013-02-21","salesAmt":"2302392559","salesShare":"30.0","salesInten":"-93876032","salesChange":"-3.9","salesAcc":"17105407905","audiCnt":"294841","audiInten":"-12964","audiChange":"-4.2","audiAcc":"2283354","scrnCnt":"604","showCnt":"2707"},{"rnum":"2","rank":"2","rankInten":"0","rankOldAndNew":"OLD","movieCd":"20127593","movieNm":"7번방의 선물","openDt":"2013-01-23","salesAmt":"1943597500","salesShare":"25.3","salesInten":"-222198500","salesChange":"-10.3","salesAcc":"82184894170","audiCnt":"265741","audiInten":"-33121","audiChange":"-11.1","audiAcc":"11491601","scrnCnt":"577","showCnt":"2479"},{"rnum":"3","rank":"3","rankInten":"0","rankOldAndNew":"OLD","movieCd":"20123641","movieNm":"잭 더 자이언트 킬러","openDt":"2013-02-28","salesAmt":"1480570000","salesShare":"19.3","salesInten":"-294694

In [ ]:
# encoding도 됨 (쓸 일이 있을까?)

json_test = json_utf8.encode()
print(json_test)

In [24]:
# json_utf8의 타입 확인하기

print(type(json_utf8))

<class 'str'>


In [32]:
# 10. json.loads(딕셔너리 형태의 문자열)

## json(딕셔너리) 형태로 적힌 문자열 자료를 딕셔너리로 변환
## 파이썬에서 사용할 수 있는 자료로 바꿀 수 있음.

json.loads(json_utf8)

{'boxOfficeResult': {'boxofficeType': '일별 박스오피스',
  'showRange': '20130302~20130302',
  'dailyBoxOfficeList': [{'rnum': '1',
    'rank': '1',
    'rankInten': '0',
    'rankOldAndNew': 'OLD',
    'movieCd': '20124401',
    'movieNm': '신세계',
    'openDt': '2013-02-21',
    'salesAmt': '2302392559',
    'salesShare': '30.0',
    'salesInten': '-93876032',
    'salesChange': '-3.9',
    'salesAcc': '17105407905',
    'audiCnt': '294841',
    'audiInten': '-12964',
    'audiChange': '-4.2',
    'audiAcc': '2283354',
    'scrnCnt': '604',
    'showCnt': '2707'},
   {'rnum': '2',
    'rank': '2',
    'rankInten': '0',
    'rankOldAndNew': 'OLD',
    'movieCd': '20127593',
    'movieNm': '7번방의 선물',
    'openDt': '2013-01-23',
    'salesAmt': '1943597500',
    'salesShare': '25.3',
    'salesInten': '-222198500',
    'salesChange': '-10.3',
    'salesAcc': '82184894170',
    'audiCnt': '265741',
    'audiInten': '-33121',
    'audiChange': '-11.1',
    'audiAcc': '11491601',
    'scrnCnt': '57

In [ ]:
# api에서 매번 데이터를 받아서 쓰면 문제는?

## 사용자가 많이 몰리면 그때그때 주게됨. 당연히 부담됨
## 기업에서는 자체 DB에 집어 넣어서, 소비자에게 꺼내서 주는 형식을 사용. 파이프라인을 구축해서 그때그때 집어넣어줌.

## << 연습 문제 >>

- 1. 일별 박스 오피스에서 여러분들이 원하는 날짜의 json 데이터를 받아서 decode(utf-8)까지 해주세요.
    - 단, 1위 ~ 3위까지만 받아옵니다.

- 2. 받아왔다면 해당 1~3위 영화중, 마음에 드는 영화를 하나 골라서 영화 상세정보 탭에서 상세 정보를 조회해주세요

- 3. 조회된 데이터를 분석해서 1:1 채팅으로 보내주시면 됩니다.

In [39]:
# 1.

key = "f5eef3421c602c6cb7ea224104795888"
target_dt = "20130302"

request_url = "http://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/\
searchDailyBoxOfficeList.json?key=%s&targetDt=%s" %(key, target_dt)

result = urllib.request.urlopen(request_url)

json_raw = result.read()

json_utf8 = json_raw.decode("utf-8")

json.loads(json_utf8)

json.loads(json_utf8)['boxOfficeResult']['dailyBoxOfficeList'][0:3]

{'boxOfficeResult': {'boxofficeType': '일별 박스오피스',
  'showRange': '20130302~20130302',
  'dailyBoxOfficeList': [{'rnum': '1',
    'rank': '1',
    'rankInten': '0',
    'rankOldAndNew': 'OLD',
    'movieCd': '20124401',
    'movieNm': '신세계',
    'openDt': '2013-02-21',
    'salesAmt': '2302392559',
    'salesShare': '30.0',
    'salesInten': '-93876032',
    'salesChange': '-3.9',
    'salesAcc': '17105407905',
    'audiCnt': '294841',
    'audiInten': '-12964',
    'audiChange': '-4.2',
    'audiAcc': '2283354',
    'scrnCnt': '604',
    'showCnt': '2707'},
   {'rnum': '2',
    'rank': '2',
    'rankInten': '0',
    'rankOldAndNew': 'OLD',
    'movieCd': '20127593',
    'movieNm': '7번방의 선물',
    'openDt': '2013-01-23',
    'salesAmt': '1943597500',
    'salesShare': '25.3',
    'salesInten': '-222198500',
    'salesChange': '-10.3',
    'salesAcc': '82184894170',
    'audiCnt': '265741',
    'audiInten': '-33121',
    'audiChange': '-11.1',
    'audiAcc': '11491601',
    'scrnCnt': '57

In [38]:
# 딕셔너리 타입의 인덱싱을 해야겠지!!!

type(json.loads(json_utf8))

dict

In [51]:
json.loads(json_utf8)['boxOfficeResult']['dailyBoxOfficeList'][0:3]

[{'rnum': '1',
  'rank': '1',
  'rankInten': '0',
  'rankOldAndNew': 'OLD',
  'movieCd': '20124401',
  'movieNm': '신세계',
  'openDt': '2013-02-21',
  'salesAmt': '2302392559',
  'salesShare': '30.0',
  'salesInten': '-93876032',
  'salesChange': '-3.9',
  'salesAcc': '17105407905',
  'audiCnt': '294841',
  'audiInten': '-12964',
  'audiChange': '-4.2',
  'audiAcc': '2283354',
  'scrnCnt': '604',
  'showCnt': '2707'},
 {'rnum': '2',
  'rank': '2',
  'rankInten': '0',
  'rankOldAndNew': 'OLD',
  'movieCd': '20127593',
  'movieNm': '7번방의 선물',
  'openDt': '2013-01-23',
  'salesAmt': '1943597500',
  'salesShare': '25.3',
  'salesInten': '-222198500',
  'salesChange': '-10.3',
  'salesAcc': '82184894170',
  'audiCnt': '265741',
  'audiInten': '-33121',
  'audiChange': '-11.1',
  'audiAcc': '11491601',
  'scrnCnt': '577',
  'showCnt': '2479'},
 {'rnum': '3',
  'rank': '3',
  'rankInten': '0',
  'rankOldAndNew': 'OLD',
  'movieCd': '20123641',
  'movieNm': '잭 더 자이언트 킬러',
  'openDt': '2013-02-28

In [54]:
# 2.

key = "f5eef3421c602c6cb7ea224104795888"
movie_cd = "20124401"

request_url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json?key=%s&movieCd=%s" % (key, movie_cd)
print(request_url)

result = urllib.request.urlopen(request_url)

json_raw = result.read()

json_utf8 = json_raw.decode("utf-8")

json.loads(json_utf8)

http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json?key=f5eef3421c602c6cb7ea224104795888&movieCd=20124401


{'movieInfoResult': {'movieInfo': {'movieCd': '20124401',
   'movieNm': '신세계',
   'movieNmEn': 'New World',
   'movieNmOg': '',
   'showTm': '134',
   'prdtYear': '2012',
   'openDt': '20130221',
   'prdtStatNm': '개봉',
   'typeNm': '장편',
   'nations': [{'nationNm': '한국'}],
   'genres': [{'genreNm': '범죄'}, {'genreNm': '액션'}],
   'directors': [{'peopleNm': '박훈정', 'peopleNmEn': 'PARK Hoon-jung'}],
   'actors': [{'peopleNm': '이정재',
     'peopleNmEn': 'LEE Jung-jae',
     'cast': '이자성',
     'castEn': 'LEE Ja-sung'},
    {'peopleNm': '최민식',
     'peopleNmEn': 'CHOI Min-shik',
     'cast': '강과장',
     'castEn': 'Detective KANG'},
    {'peopleNm': '황정민',
     'peopleNmEn': 'HWANG Jung-min',
     'cast': '정청',
     'castEn': 'JUNG Chung'},
    {'peopleNm': '박성웅',
     'peopleNmEn': 'PARK Sung-woong',
     'cast': '이중구',
     'castEn': ''},
    {'peopleNm': '송지효',
     'peopleNmEn': 'SONG Ji-hyo',
     'cast': '신우',
     'castEn': ''},
    {'peopleNm': '정영기',
     'peopleNmEn': 'JEONG Young-ki'

In [55]:
result = urllib.request.urlopen(request_url)

json_raw = result.read()

json_utf8 = json_raw.decode("utf-8")

json.loads(json_utf8)

{'movieInfoResult': {'movieInfo': {'movieCd': '20124401',
   'movieNm': '신세계',
   'movieNmEn': 'New World',
   'movieNmOg': '',
   'showTm': '134',
   'prdtYear': '2012',
   'openDt': '20130221',
   'prdtStatNm': '개봉',
   'typeNm': '장편',
   'nations': [{'nationNm': '한국'}],
   'genres': [{'genreNm': '범죄'}, {'genreNm': '액션'}],
   'directors': [{'peopleNm': '박훈정', 'peopleNmEn': 'PARK Hoon-jung'}],
   'actors': [{'peopleNm': '이정재',
     'peopleNmEn': 'LEE Jung-jae',
     'cast': '이자성',
     'castEn': 'LEE Ja-sung'},
    {'peopleNm': '최민식',
     'peopleNmEn': 'CHOI Min-shik',
     'cast': '강과장',
     'castEn': 'Detective KANG'},
    {'peopleNm': '황정민',
     'peopleNmEn': 'HWANG Jung-min',
     'cast': '정청',
     'castEn': 'JUNG Chung'},
    {'peopleNm': '박성웅',
     'peopleNmEn': 'PARK Sung-woong',
     'cast': '이중구',
     'castEn': ''},
    {'peopleNm': '송지효',
     'peopleNmEn': 'SONG Ji-hyo',
     'cast': '신우',
     'castEn': ''},
    {'peopleNm': '정영기',
     'peopleNmEn': 'JEONG Young-ki'

In [ ]:
# 3.

<신세계>

영화 코드 : 20124401
영화 명 : 신세계(New World)
상영시간 : 134분
제작연도 : 2012
개봉연도 : 2013년 02월 21일
제작 상태 : 개봉
영화 유형 : 장편
제작 국가 : 한국
장르 : 범죄, 액션
감독 : 박훈정
배우 : 이정재(이자성), 최민식(강과장) .. 등
상영형태 : 필름, 2D, 디지털
참여 영화사 : (주)사나이픽처스, (주)페퍼민트앤컴퍼니 .. 등
심의 정보 : 청소년 관람 불가
스태프 : 김주형, 김형철, 여한구 ..

### << 추가 문제 >>

- vscode나 스프링 프로젝트에 하나 더 추가해서

- 1. 날짜를 폼으로 입력 받고 제출 버튼을 누르면
    - date로 받은 데이터를 SimpleDateFormat 자료형으로 yyyymmdd 형식으로 바꿔서 전달해야합니다
- 2. getJSON을 이용해서 해당 날짜의 일일 박스오피스를 화면에 띄워주세요.